# # Generating Fits with linear parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [ ]:
# Initial parameters
params = {
    'I0': 2.0,
    'gamma': 1.2,
    'r_g': 1,
    'sigma': 0.2,
    'f': 0.6
}

# Steps for linear increase
I0_step = 2
gamma_step = 0
r_g_step = 0
sigma_step = 0
f_step = 0

# Numbers of profiles to generate
num_profiles = 5

### Synthetic Profile

In [ ]:
# Model definition
r = np.linspace(0.1, 3, 300)

# Initialize storage list
results = []


# The model function
def model(r, I0, gamma, r_g, sigma, f):
    c = 0.3
    R_cutL = 0.1  # Fixed
    R_cutR = 3.0  # Fixed

    r_sinc = c * (np.pi * ((r - r_g) / sigma))**2

    powerlaw = I0 * ((r / r_g)**(-gamma))
    sinc = np.sin(r_sinc) / r_sinc
    gauss = np.exp((-(r - r_g)**4) / (2 * sigma**4))
    L_cut_factor = (r >= R_cutL).astype(float)
    R_cut_factor = (r <= R_cutR).astype(float)
    # NO_SINC_MODEL
    gap_factor = (1 - f * (gauss))

    return powerlaw * gap_factor * L_cut_factor * R_cut_factor


# Image generation from the profile
def generate_image_from_profile(radius, intensity, image_size=256):
    """
    Generates a 2D image from a 1D radial intensity profile.

    Parameters:
    radius (array-like): 1D array of radius values.
    intensity (array-like): 1D array of intensity values corresponding to the radius.
    image_size (int): Size of the output image (image_size x image_size).

    Returns:
    np.ndarray: 2D array representing the generated image.
    """
    # Ensure radius and intensity are numpy arrays
    radius = np.asarray(radius)
    intensity = np.asarray(intensity)

    # Check if radius and intensity have the same length
    if radius.shape != intensity.shape:
        raise ValueError("Radius and intensity must have the same shape.")

    y, x = np.indices((image_size, image_size))
    center = (image_size - 1) / 2
    r_img = np.sqrt((x - center)**2 + (y - center)**2)

    # Rescale so that max pixel radius = max profile radius
    r_img = r_img / r_img.max() * radius.max()

    # Interpolate profile onto the pixel grid
    image = np.interp(r_img, radius, intensity, left=0, right=0)
    return image

In [ ]:
for i in range(1, num_profiles + 1):

    # Generate new parameters from the old ones
    I0_init = params.get("I0_init") + I0_step
    gamma_init = params.get("gamma_init") + gamma_step
    r_g_init = params.get("r_g_init") + r_g_step
    sigma_init = params.get("sigma_init") + sigma_step
    f_init = params.get("f_init") + f_step

    print(f"Profile {i}: I0={I0_init:.2f}, gamma={gamma_init:.2f}, r_g={r_g_init:.2f}, sigma={sigma_init:.2f}, f={f_init:.2f}")

    # Profile generation
    profile = model(r, I0_init, gamma_init, r_g_init, sigma_init, f_init)

    # Graph
    plt.figure(figsize=(8, 5))
    plt.plot(r, profile, label="Example Model", color="black", linestyle="-", linewidth=1)
    plt.xlabel("Radius [au]")
    plt.ylabel("Intensity")
    plt.ylim(0, 2 * np.max(model(r, 1.0, 1.0, 0.65, 0.1, 0.5)))
    plt.legend()
    plt.grid(True)
    plt.show()

    # Create 2D
    image = generate_image_from_profile(r, profile, image_size=128)
    plt.imshow(image, origin='lower', cmap='grey')
    plt.colorbar(label='Intensity')
    plt.title('2D Disk Image')
    plt.xlabel('Pixel X')
    plt.ylabel('Pixel Y')
    plt.show()

    # Save as a FITS file
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(f'Generated_Fits/Generated_Fits_Linear/synthetic_disk_{i}.fits', overwrite=True)
    results.append([params, fig1, fig2, fits_filename])